In [2]:
import os
os.chdir('../')

import sys
sys.argv=['']
del sys

import argparse
import json
import matplotlib.pyplot as plt
import numpy as np
import optuna
import pandas as pd
import random
import scipy

from sklearn.preprocessing import StandardScaler
from sksurv.util import Surv as skSurv
from sksurv.metrics import concordance_index_ipcw, cumulative_dynamic_auc, brier_score, integrated_brier_score
from sklearn_pandas import DataFrameMapper
from sklearn.utils import resample
from utils.param_search import *
from utils.output_results import *

The uncertainty measure used here is the Deep Ensemble method.

2 real datasets can be used : 
- the LungCancerExplorer dataset is a set composed of multiple sources of data
- the METABRIC cohort.

We apply this method to different types of neural network models :
- CoxCC, CoxTime
- DeepHit

In [3]:
parser = argparse.ArgumentParser()
parser.add_argument('--dataset', '-dt', default='Metabric', type=str) #LungCancerExplorer, Metabric
parser.add_argument('--name', '-n',type=str, default="DeepHit")#CoxTime, DeepHit, CoxCC 
parser.add_argument('--plot_mode', '-pm', default=False, action='store_true')
parser.add_argument('--timepoints', '-tp',type=str, default="fixed") #fixed, percentiles
parser.add_argument('--uncertainty', '-u',type=str, default="DeepEnsemble") #Bootstrap, MCDropout, DeepEnsemble, VAE, BMask
config = parser.parse_args()

In [4]:
dir_res = 'results/'+ config.dataset + "/"+config.uncertainty+"/"+ config.name+'/'
os.makedirs(dir_res, exist_ok=True)

dir_data = "data/" + config.dataset + "/"

# Data Preprocessing

Missing data are previously handled using the MICE package in R.

In [5]:
df = pd.read_csv(dir_data+"DataImputed.csv")
df['id'] = df.index + 1

We list the clinical variables and the genes variables according to the dataset considered.

In [6]:
if config.dataset == "Metabric":
    ClinVar =['age',
                   'chemotherapy',
                   'grade1',
                   'grade2',
                   'hormonotherapy',
                   'N',
                   'tumor_size']
    ColsLeave = ['chemotherapy',
                  'grade1',
                  'grade2',
                  'id',
                  'hormonotherapy',
                  'N',
                  'status',
                  'yy']
elif config.dataset == "LungCancerExplorer":
    ClinVar = ['Pat_Age',
                    'Pat_Stage_II',
                    'Pat_Stage_III',
                    'Pat_Stage_IV',
                    'Pat_Stage_I_or_II']
    ColsLeave = ['id',
                  'Pat_Stage_II',
                  'Pat_Stage_III',
                  'Pat_Stage_IV',
                  'Pat_Stage_I_or_II',
                  'status',
                  'yy']
GenesVar = df.drop(columns = ClinVar + ['id','status','yy'],axis=1).columns.to_list()

The 5 folds of the simple cross-validation are defined and saved beforehand in order to always have the same folds between the different methods.

In [7]:
with open(dir_data+"folds_1CV.json") as f:
    kfolds = json.load(f)

In [8]:
df_train = df[ClinVar+GenesVar+['yy','status','id']].loc[kfolds['train']] 
df_test = df[ClinVar+GenesVar+['yy','status','id']].loc[kfolds['test']] 
AllVar = ClinVar+GenesVar

The continuous variable are standardized. The yy variable corresponds to the survival time, the status is the censoring indicator (a value of 0 corresponds to censoring) and the id variable is the id of the patient.

In [9]:
ColsStandardize = [col for col in AllVar if col not in ColsLeave] 
standardize = [([col], StandardScaler()) for col in ColsStandardize]
leave = [(col, None) for col in ColsLeave]
df_mapper = DataFrameMapper(standardize + leave, df_out=True) 

# Hyperparameter Search

A simple 5-folds cross validation is implemented using the training set to determine the hyperparameters of the neural network models. The optuna package is used to perform the hyperparmeter search. The hyperparameter that are searched are the following:

| Hyperparameter | Values |
|----------|--------------|
| Activation function |  {tanh, relu} |
| Batch size |  {8,16,32,64,128} |
| Dropout rate |  [0.0,0.3] | 
|Layers | {1,2,3,4}|
|Learning rate|[1e-3, 1e-2]|
|Neurons|[4,128]|
|Optimizer|{adam, adam_amsgrad, RMSProp, SGDWR}|
|Pénalisation L2|[0,0.1]|
|Alpha (DeepHit)|[0,1]|
|Sigma(DeepHit)|{0.1,0.25,0.5,1,2.5,5,10,100}|
|Durations(DeepHit)|{10,50,100,200,400}|

In [10]:
sampler = optuna.samplers.TPESampler()
study = optuna.create_study(study_name = config.name, 
                            storage = 'sqlite:///'+dir_res+config.name+'.db',
                            sampler=sampler, 
                            direction='minimize', 
                            load_if_exists=True)


study.optimize(lambda trial : objective_net(trial,
                                            df_train,
                                            df_mapper,
                                            dir_res,
                                            config), 
                                           n_trials=200)
trial = study.best_trial
outer_loop = pd.DataFrame([trial.params])
outer_loop.to_csv(dir_res+'best_param.csv', sep = ';', index = False, header = True)
df_results = study.trials_dataframe()
df_results.to_csv(dir_res+'trials_dataframe.csv', sep = ';', header = True)

fig = optuna.visualization.plot_optimization_history(study)
fig.write_image(dir_res+"optim_history.png")
fig2 = optuna.visualization.plot_slice(study)
fig2.write_image(dir_res+"parameters_history.png")

[I 2022-02-02 11:21:26,679] A new study created in RDB with name: DeepHit
[I 2022-02-02 11:23:02,845] Trial 0 finished with value: 1.5768827438354491 and parameters: {'activation': 'tanh', 'batch_size': 16, 'n_layers': 4, 'learning_rate': 0.002314845462333993, 'neurons': 96, 'optimizer': 'adam_amsgrad', 'l2': 0.042175745590026105, 'dropout': 0.15568239823266364, 'num_durations': 400, 'alpha': 0.4668119737058778, 'sigma': 10}. Best is trial 0 with value: 1.5768827438354491.
[I 2022-02-02 11:23:49,471] Trial 1 finished with value: 0.34262646436691285 and parameters: {'activation': 'tanh', 'batch_size': 64, 'n_layers': 1, 'learning_rate': 0.0020370902234228662, 'neurons': 78, 'optimizer': 'adam_amsgrad', 'l2': 0.0678533578408319, 'dropout': 0.2475888391514687, 'num_durations': 10, 'alpha': 0.10008811218478098, 'sigma': 5}. Best is trial 1 with value: 0.34262646436691285.
[I 2022-02-02 11:24:40,792] Trial 2 finished with value: 0.9628446817398071 and parameters: {'activation': 'relu', 'bat

ValueError: Expected more than 1 value per channel when training, got input size torch.Size([1, 12])

# Uncertainty Measure using DeepEnsemble

We build the model using always the same hyperparameters that were selected by cross validaton. Then, the model is trained M times, randomly initializing the weights each time. The training set is randomly shuffled. The training criterion is the loss. M predictions per point of the test set are outputed. This is the method called [DeepEnsemble](https://proceedings.neurips.cc/paper/2017/file/9ef2ed4b7fd2c810847ffa5fa85bce38-Paper.pdf).

We load the hyperparameters selected previously.

In [ ]:
outer_loop = pd.read_csv(dir_res+'best_param.csv', sep = ';')
config.acti_func = outer_loop['activation'][0]
config.batch_size = outer_loop['batch_size'][0]
config.dr = outer_loop['dropout'][0]
config.layers = outer_loop['n_layers'][0]
config.lr  = outer_loop['learning_rate'][0]
config.neurons = outer_loop['neurons'][0]
config.optim = outer_loop['optimizer'][0]
config.pen_l2 = outer_loop['l2'][0]

if config.name=="DeepHit":
    config.alpha = outer_loop['alpha'][0] 
    config.sigma = outer_loop['sigma'][0]
    config.num_durations = outer_loop['num_durations'][0]
    labtrans = DeepHitSingle.label_transform(config.num_durations)
elif config.name=="CoxTime":
    labtrans = CoxTime.label_transform()
else:
    labtrans=""

We define the number of repetitions M.

In [ ]:
M=100

We sample 20\% of the train set and define it as a validation set.

In [ ]:
df_val = df_train.sample(frac=0.2, random_state = 1234)
df_train = df_train.drop(df_val.index)
df_train = df_mapper.fit_transform(df_train)
df_val = df_mapper.transform(df_val).astype('float32')
df_test = df_mapper.transform(df_test).astype('float32')

If the timepoints are defined as percentiles, the results are outputed at the percentiles of survival times. If the timepoints are deifned as fixed, teh results are outputed at specific times.

For the breast cancer dataset, the fixed timepoints used are at 5 and 10 years. For the LungCancerExplorer, we output the measures at 2 and 5 years. 

In [ ]:
if config.timepoints == "percentiles":
    kmf = KaplanMeierFitter()
    kmf.fit(np.array(df_train['yy']), np.array(df_train['status']))
    time_grid = np.array(kmf.percentile(np.linspace(0.9, 0.1, 9)).iloc[:,0])
elif config.timepoints == "fixed":
    if config.dataset == "Metabric":
        time_grid = [2,5]
    elif config.dataset == "LungCancerExplorer":
        time_grid = [24,60]
    else:
        time_grid = [0.5,1,2,3]

We define x and y for the train set, the validation set and the test set.

In [ ]:
x_train = np.array(df_train.drop(['yy','status','id'], axis=1)).astype('float32')
x_val = np.array(df_val.drop(['yy','status','id'], axis=1)).astype('float32')
x_test = np.array(df_test.drop(['yy','status','id'], axis=1)).astype('float32')
y_train = (df_train['yy'].values, df_train['status'].values)
y_val = (df_val['yy'].values, df_val['status'].values)
y_test = (df_test['yy'].values, df_test['status'].values)
    
if labtrans !="":
    y_train = labtrans.fit_transform(*y_train)
    y_val = labtrans.transform(*y_val)

val = (x_val, y_val)
in_features = x_train.shape[1]

We compute the Concordance Index (CAll), the Brier Score (BSAll) and the survival predictions for all the patients of the test set (PredAll) for each timepoint of the time grid previously defined.

In [ ]:
CAll = pd.DataFrame()
BSAll = pd.DataFrame()
PredAll = []
measures = pd.DataFrame()

for j in range(M):
    print(j)
    model,callbacks = build_model_net(config,
                                  in_features,
                                  labtrans)
    log = model.fit(x_train, 
                    y_train, 
                    int(config.batch_size),
                    epochs = 500, 
                    callbacks = callbacks,
                    verbose = False,
                    val_data = val,
                   shuffle = True)

    if config.name in ["CoxCC","CoxTime"]:
        _ = model.compute_baseline_hazards()
        surv = model.predict_surv_df(x_test)
    elif config.name == "DeepHit":
         surv = model.interpolate(10).predict_surv_df(x_test)

    data_train = skSurv.from_arrays(event=df_train['status'], time=df_train['yy'])
    data_test = skSurv.from_arrays(event=df_test['status'], time=df_test['yy'])
    CAll[j] = [concordance_index_ipcw(data_train, data_test, np.array(-determine_surv_prob(surv,t)),t)[0] for t in time_grid]
    BSAll[j] = [brier_score(data_train, data_test, np.array(-determine_surv_prob(surv,t)),t)[1][0] for t in time_grid]
    preds = np.asarray([determine_surv_prob(surv,t) for t in time_grid])
    PredAll.append(preds)  
    del model 
    del log

In [ ]:
measures['C'] = CAll.mean(axis=1)
measures['BS'] = BSAll.mean(axis=1)
measures['Time'] = time_grid
measures.to_csv(dir_res+'measures_'+config.name+'.csv', sep = ';', header = True, index=True)

In [ ]:
measures

We output survival prediction intervals for a certain number of patients (n_pat) for each timepoint of the timegrid.

In [ ]:
n_pat = 3

In [ ]:
random.seed(4893)
patients_test = random.sample(list(df_test['id']),n_pat)
ResPat = []
for t in range (len(time_grid)):
    ResTime = pd.DataFrame([PredAll[l][t] for l in range(M)]).T
    for i in patients_test:
        ResTime.index = df_test['id']
        ic = output_ic(ResTime.loc[i,:],0.95)
        values = df_test.loc[:,['status','yy','id']][df_test['id']==i]
        ResPat.append(values.values.tolist()[0]+list(ic))
ResPat = pd.DataFrame(ResPat)
ResPat.columns = ['status','yy','id','IClow','ICmean','IChigh']
ResPat["Time"] = np.repeat(time_grid,n_pat)
ResPat.to_csv(dir_res+"surv_intervals.csv", sep=';', header = True,index=False)

In [ ]:
ResPat